In [5]:
import pandas as pd

# grade

In [6]:
g4 = pd.read_excel('./数据处理/grade/4.grades_TsinghuaX-20220332_1X-_.xlsx')
g5 = pd.read_excel('./数据处理/grade/5.grades_TsinghuaX-20220332_2X-_.xlsx')
g6 = pd.read_excel('./数据处理/grade/6.grades_TsinghuaX-30240184_1X-_.xlsx')
g8 = pd.read_excel('./数据处理/grade/8.grades_TsinghuaX-80000901_1X-_.xlsx')
g10 = pd.read_excel('./数据处理/grade/10.grades_TsinghuaX-80512073_2014_1X-_.xlsx')
g11 = pd.read_excel('./数据处理/grade/11.grades_TsinghuaX-80512073X-_.xlsx')

In [7]:
import re
def helper(x):
    quiz_avg = None
    quiz_count = 0
    if 'Quiz Avg' in x.columns:
        quiz_avg = x['Quiz Avg']
        quiz_count = len([c for c in x.columns if re.findall('Quiz \d+', c)])
    elif 'HW Avg' in x.columns:
        quiz_avg = x['HW Avg']
        quiz_count = len([c for c in x.columns if re.findall('HW \d+', c)])
    elif '作业总评' in x.columns:
        quiz_cols = [c for c in x.columns if c.endswith('习题')]

        quiz_avg = x[quiz_cols].mean(axis=1)
        quiz_count = len(quiz_cols)

    elif '加分' in x.columns:
        # quiz_cols = [c for c in x.columns if re.findall('PS|PA \d+', 'PA 01')]
        quiz_cols = [c for c in x.columns if re.findall('PS|PA \d+', c)]
        quiz_avg = x[quiz_cols].mean(axis=1)
        quiz_count = len(quiz_cols)
    return quiz_avg, quiz_count

In [ ]:

courseid_list = ['TsinghuaX/20220332_1X/_', 'TsinghuaX/20220332_2X/_', 'TsinghuaX/30240184_1X/_',
 'TsinghuaX/80000901_1X/_', 'TsinghuaX/80512073_2014_1X/_2014_', 'TsinghuaX/80512073X/_']
grades = []
for i, x in enumerate([g4, g5, g6, g8, g10, g11]):
    print(x.shape)
    col1, col2 = 'ID', 'Total'
    if '总分(百分制）' in x.columns:
        col2 = '总分(百分制）'
    if '总分' in x.columns:
        col2 = '总分'
    if 'user ID' in x.columns:
        col1 = 'user ID'
    _df = x[[col1, col2]]
    _df.columns = ['userid', 'total']
    _df['courseid'] = courseid_list[i]
    quiz_avg, quiz_count = helper(x)
    _df['quiz_avg'] = quiz_avg
    _df['quiz_count'] = quiz_count
    grades.append(_df)


In [9]:
grade_df = pd.concat(grades)
grade_df['studentid'] = list(range(grade_df.shape[0]))
grade_df.head()

,userid,total,courseid,quiz_avg,quiz_count,studentid
0,7653,99.000000,TsinghuaX/20220332_1X/_,1.000000,45,0
1,4069,92.500000,TsinghuaX/20220332_1X/_,1.000000,45,1
2,19624,91.611111,TsinghuaX/20220332_1X/_,0.988889,45,2
3,7580,90.507122,TsinghuaX/20220332_1X/_,0.957407,45,3
4,8104,89.403846,TsinghuaX/20220332_1X/_,1.000000,45,4


# log forum

In [16]:
log = pd.read_excel('./数据处理/log.xlsx')
forum = pd.read_excel('./数据处理/forum.xlsx')

/root/miniconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [17]:
def count_log_series(series):
    count = len(series[series!=0])
    return count
# 按 userid 和 course 分组，并对 video 和 assignment 求和
# 对组内的video和assignment求和并统计它们的次数
agg_dict = {"video": ["sum", count_log_series], "assignment": ["sum", count_log_series]}
result = log.groupby(["userid", "course"]).agg(agg_dict)

# 重命名聚合后的列名
result.columns = ["_".join(x) for x in result.columns.ravel()]

# 重置索引以使组名成为列
log_dataset = result.reset_index()
# log_dataset['studentid'] = list(range(log_dataset.shape[0]))
log_dataset.columns = ['userid', 'courseid', 'video_sum', 'video_count', 'assignment_sum',
       'assignment_count']
log_dataset.head()

/tmp/ipykernel_983/3002759336.py:10: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  result.columns = ["_".join(x) for x in result.columns.ravel()]


,userid,courseid,video_sum,video_count,assignment_sum,assignment_count
0,1,TsinghuaX/00690242_1X/_,72.815582,5,1.519474,1
1,1,TsinghuaX/20220332_1X/_,0.000000,0,848.274412,3
2,1,TsinghuaX/80000901_1X/_,334.503000,1,0.000000,0
3,4,TsinghuaX/00690242_1X/_,96.214645,3,12.124429,1
4,4,TsinghuaX/00720091X/_,1587.883000,2,14.567349,1


In [18]:
# 按照userid和courseid分组，对body进行计数
result = forum.groupby(['userid', 'courseid'])['body'].count()
forum_dataset = result.reset_index()
forum_dataset.columns = ['userid', 'courseid', 'forum_count']
forum_dataset.head()

,userid,courseid,forum_count
0,1,tsinghuax/cs001/test,2
1,2,tsinghuax/cs001/test,5
2,5,CS/CPP/C_,1
3,5,CS/CSCareer/_,4
4,5,CS/TsinghuaX/TsinghuaX_,5



# 聚合、

In [ ]:
df = grade_df.merge(dataset_df, on=['userid', 'courseid'], how='left').merge(log_dataset, on=['userid', 'courseid'], how='left').merge(forum_dataset, on=['userid', 'courseid'], how='left')

In [ ]:
df

In [ ]:
df = df[['studentid','userid', 'courseid', 'video_count', 'video_sum',
      'assignment_count', 'assignment_sum', 'forum_count', 'quiz_avg', 'quiz_count','total']]
df['learning_achivement'] = (df['total']<60)+0


In [ ]:
df.to_excel('./count-result-final.xlsx', index=False)

In [ ]:
df = df.dropna(subset=['video_count', 'video_sum',
      'assignment_count', 'assignment_sum', 'forum_count'], how='all')
df = df.fillna(value=0)

In [ ]:

df.to_excel('./count-result-final-drop-space.xlsx', index=False)

In [ ]:
df

# plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
def six_course_count(col):
    curr_df = df[['courseid'] + [col]]
    if col == 'studentid':
        count = curr_df.groupby('courseid').size()
    else:
        count = curr_df.groupby('courseid').sum()
    return count

# df = pd.read_excel('./count-result-final-drop-space.xlsx')
df = pd.read_excel('./59581.xlsx')
# 可以选这几个 studentid video_count assignment_count forum_count
col = 'video_count'
count = six_course_count(col)
count.plot(kind='bar')        
plt.ylabel(col)